In [1]:
import os
import random
import pandas as pd
from joblib import dump, load
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime

from preprocessing import TimeSeriesPreprocessor
from SCINet import SCINet, StackedSCINet

In [2]:
def make_stackedSciNet(input_shape):
    inputs = tf.keras.Input(shape=(input_shape[1], input_shape[2]), name='inputs')
    x = StackedSCINet(horizon, features=input_shape[-1], stacks=K, levels=L, h=h,
                      kernel_size=kernel_size, dropout=dropout)(inputs)
    model = tf.keras.Model(inputs, x)

    model.summary()
    tf.keras.utils.plot_model(model, to_file='modelDiagram.png', show_shapes=True)

    return model


def make_sciNet(input_shape):
    inputs = tf.keras.Input(shape=(input_shape[1], input_shape[2]), name='inputs')
    x = SCINet(horizon, features=input_shape[-1], levels=L, h=h,
               kernel_size=kernel_size, dropout=dropout)(inputs)
    model = tf.keras.Model(inputs, x)

    model.summary()
    tf.keras.utils.plot_model(model, to_file='modelDiagram.png', show_shapes=True)

    return model

In [3]:
data_filepath = 'datasets/ETDataset-main/ETT-small/ETTh1.csv'
y_col = 'OT'
index_col = 'date'

# Hyperparams
degree_of_differencing = 0
# T, tilta
look_back_window, horizon = 96, 48
batch_size = 16
learning_rate = 9e-3
h, kernel_size, L, K = 4, 5, 3, 1
l1, l2 = 0, 0
dropout = 0.25
# split_strides = look_back_window + horizon
split_strides = 1

In [4]:

# Load and preprocess data
data = pd.read_csv(data_filepath, index_col=index_col).astype('float32')
data.index = pd.to_datetime(data.index)

train_data = data[:int(0.6 * len(data))]
val_data = data[int(0.6 * len(data)):int(0.8 * len(data))]
test_data = data[int(0.8 * len(data)):]

# # Get the minimum and maximum dates in the DataFrame
# min_date = data.index.min()
# max_date = data.index.max()
#
# # Calculate the splitting points
# train_end_date = min_date + pd.DateOffset(months=12) - pd.DateOffset(days=1)
# val_end_date = train_end_date + pd.DateOffset(months=4)
# test_end_date = val_end_date + pd.DateOffset(months=4)
#
# # Split into train, val, and test sets based on the calculated dates
# train_data = data[data.index <= train_end_date]
# val_data = data[(data.index > train_end_date) & (data.index <= val_end_date)]
# test_data = data[(data.index > val_end_date) & (data.index <= test_end_date)]


In [5]:
# Train model
preprocessor = TimeSeriesPreprocessor(look_back_window, horizon, split_strides, degree_of_differencing,
                                      relative_diff=False, scaling='standard')
X_train, y_train = preprocessor.fit_transform(train_data)
X_val, y_val = preprocessor.transform(val_data)
print(f'Input shape: X{X_train.shape}, y{y_train.shape}')

model = make_stackedSciNet(X_train.shape)

Input shape: X(10309, 96, 7), y(10309, 48, 7)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 96, 7)]           0         
                                                                 
 stacked_sci_net (StackedSC  (None, 48, 7)             281988    
 INet)                                                           
                                                                 
Total params: 281988 (1.08 MB)
Trainable params: 281988 (1.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [6]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = f'logs/gradient_tape/{current_time}/train'
val_log_dir = f'logs/gradient_tape/{current_time}/val'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
val_summary_writer = tf.summary.create_file_writer(val_log_dir)


In [7]:
from tqdm import tqdm

loss_fn = tf.keras.losses.MeanSquaredError()

# Hyperparameters
epochs = 150

# Initialize early stopping parameters
best_val_mae = float('inf')
patience = 5  # for early stopping
counter = 0
best_weights = None

# Create optimizer
# optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
# optimizer.build(model.trainable_weights)
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)

# Split data into batches
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset = dataset.shuffle(buffer_size=1024, seed=4321).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.shuffle(buffer_size=1024, seed=4321).batch(batch_size)

# MAE metrics
train_mae_metric = tf.keras.metrics.MeanAbsoluteError()
val_mae_metric = tf.keras.metrics.MeanAbsoluteError()


# Custom training loop
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Initialize tqdm with the number of steps (batches) per epoch
    prog_bar = tqdm(dataset, unit="step", leave=False)

    for step, (x_batch, y_batch) in enumerate(prog_bar):
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x_batch)

            # Forward pass
            y = model(x_batch, training=True)

            # Calculate loss
            losses = []
            for output in model.layers[1].outputs:
                loss = loss_fn(y_batch, output)
                losses.append(loss)
            total_loss = sum(losses)

            # Log training loss and MAE to TensorBoard
            with train_summary_writer.as_default():
                tf.summary.scalar('Training Loss', total_loss, step=epoch)
                tf.summary.scalar('Training MAE', train_mae_metric.result(), step=epoch)

            # Update tqdm description with current loss
            prog_bar.set_description(f"Training loss: {total_loss:.4f}")

        # Update MAE metric for training data
        train_mae_metric.update_state(y_batch, y)

        # Calculate gradients
        all_grads = []
        for loss, scinet in zip(losses, model.layers[1].scinets):
            grads = tape.gradient(loss, scinet.trainable_weights)
            all_grads.append(grads)

        # Update weights
        for grads, scinet in zip(all_grads, model.layers[1].scinets):
            optimizer.apply_gradients(zip(grads, scinet.trainable_weights))

        # Clean up resources of the tape
        del tape

    # Validation loop
    for x_val_batch, y_val_batch in val_dataset:
        val_predictions = model(x_val_batch)
        val_mae_metric.update_state(y_val_batch, val_predictions)

        val_loss = loss_fn(y_val_batch, val_predictions)

        with val_summary_writer.as_default():
            tf.summary.scalar('Validation Loss', val_loss, step=epoch)
            tf.summary.scalar('Validation MAE', val_mae_metric.result(), step=epoch)

    # Show metrics
    print(f"Training MAE: {train_mae_metric.result()}")
    print(f"Validation MAE: {val_mae_metric.result()}")

    # Early stopping check
    if val_mae_metric.result() < best_val_mae:
        best_val_mae = val_mae_metric.result()
        best_weights = model.get_weights()  # save best weights
        counter = 0
    else:
        counter += 1

    if counter >= patience:
        print("Early stopping triggered")
        model.set_weights(best_weights)  # restore best weights
        break

    # Reset metrics for the next epoch
    train_mae_metric.reset_states()
    val_mae_metric.reset_states()


Epoch 1/150


Training MAE: 0.5570996403694153
Validation MAE: 0.6282496452331543
Epoch 2/150


Training MAE: 0.4438401460647583
Validation MAE: 0.6720663905143738
Epoch 3/150


Training MAE: 0.4510931968688965
Validation MAE: 0.6420608162879944
Epoch 4/150


Training MAE: 0.4670115113258362
Validation MAE: 0.784242570400238
Epoch 5/150


Training MAE: 0.5056880712509155
Validation MAE: 1.0457717180252075
Epoch 6/150


Training MAE: 0.5536084175109863
Validation MAE: 0.7142295241355896
Early stopping triggered


In [8]:

# # Generate new id and create save directory
# existing_ids = [int(name) for name in os.listdir('saved-models/') if name.isnumeric()]
# run_id = random.choice(list(set(range(0, 1000)) - set(existing_ids)))
# save_directory = f'saved-models/regressor/{run_id:03d}/'
# os.makedirs(os.path.dirname(save_directory), exist_ok=True)
#
# # Save model, preprocessor and training history
# model.save(save_directory)
# with open(save_directory + 'preprocessor', 'wb') as f:
#     dump(preprocessor, f, compress=3)
# pd.DataFrame(history.history).to_csv(save_directory + 'train_history.csv')

# Evaluate
# run_id = 186
# model = load_model(f'saved-models/regressor/{run_id:03d}/')
# with open(f'saved-models/regressor/{run_id:03d}/preprocessor', 'rb') as f:
#     preprocessor = load(f)
X_test, y_test = preprocessor.transform(test_data)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

y_pred = model(X_test)
loss = loss_fn(y_test, y_pred)

print(f"Test loss: {loss:.4f}")

# # Save evaluation results
# if not isinstance(scores, list):
#     scores = [scores]
# row = [run_id] + scores + [pd.Timestamp.now(tz='Australia/Melbourne')]
# try:
#     df_scores = pd.read_csv('saved-models/scores.csv')
#     df_scores.loc[len(df_scores)] = row
# except (FileNotFoundError, ValueError):
#     df_scores = pd.DataFrame([row], columns=['id'] + list(model.metrics_names) + ['time'])
# df_scores.to_csv('saved-models/scores.csv', index=False)

# # Predict
# # y_test is only used to calculate loss, how to get rid of it?
# y_pred = model.predict({'inputs': X_test, 'targets': y_test})
# y_pred = preprocessor.scaler.inverse_transform(y_pred.reshape(-1, y_test.shape[-1]))
# y_test = preprocessor.scaler.inverse_transform(y_test.reshape(-1, y_test.shape[-1]))
# comparison = np.hstack([y_pred, y_test])
# df = pd.DataFrame(comparison, columns=['Predicted', 'Actual'])
# df.to_csv(f'saved-models/regressor/{run_id:03d}/comparison.csv')

Test loss: 0.7962
